In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from tqdm import tqdm

In [ ]:
class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvolutionBlock, self).__init__()

        self.convs = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=3, padding=0, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.MaxPool2d(2),
            nn.ReLU()
        )

    def forward(self, X):
        return self.convs(X)

In [ ]:
class Classifier(nn.Module):
    def __init__(self, in_channels,  num_classes) -> None:
        super(Classifier, self).__init__()

        conv1 = ConvolutionBlock(in_channels=in_channels, out_channels=8)
        conv2 = ConvolutionBlock(in_channels=8, out_channels=16)
        conv3 = ConvolutionBlock(in_channels=16, out_channels=32)

        self.layers = nn.Sequential(conv1, conv2, conv3)

        self.fc = nn.Sequential(
            nn.Linear(in_features=1152, out_features=40),
            nn.ReLU(),
            nn.Linear(in_features=40, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=num_classes),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, X):
        X = self.layers(X)
        X = X.view(X.shape[0], -1)
        X = self.fc(X)
        return X

In [ ]:
def get_dataloaders(path: str, training: bool):
    global transforms
    if training:
        transforms = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip()
        ])
    else:
        transforms = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor()
        ])

    data = torchvision.datasets.ImageFolder(
        path, transform=transforms)

    data_loader = DataLoader(data, batch_size=32, shuffle=training)

    return data_loader

In [ ]:
def train_model(model=Classifier(in_channels=3, num_classes=4)):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    EPOCHS = 30
    train_loader = get_dataloaders(path='Mask_Data/train', training=True)
    model.train()
    for epoch in tqdm(range(EPOCHS)):
        # for batch_idx, (inputs, labels) in enumerate(train_loader):
        for batch in train_loader:
            inputs, labels = batch

            # Forward pass
            y_pred = model.forward(inputs)
           # print('y_pred shape: ', y_pred.shape)

            # Calculating loss
            loss = criterion(input=y_pred, target=labels)

            # BackPropagation
            loss.backward()
            optimizer.step()

            # Zero Gradients
            optimizer.zero_grad()

        print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
    return model


model = train_model()

In [22]:
class Classifier(nn.Module):
    def __init__(self, in_channels,  num_classes) -> None:
        super(Classifier, self).__init__()

        conv1 = ConvolutionBlock(in_channels=in_channels, out_channels=8)
        conv2 = ConvolutionBlock(in_channels=8, out_channels=16)
        conv3 = ConvolutionBlock(in_channels=16, out_channels=32)

        self.layers = nn.Sequential(conv1, conv2, conv3)

        self.fc = nn.Sequential(
            nn.Linear(in_features=1152, out_features=40),
            nn.ReLU(),
            nn.Linear(in_features=40, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=num_classes),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, X):
        X = self.layers(X)
        X = X.view(X.shape[0], -1)
        X = self.fc(X)
        return X

In [23]:
def get_dataloaders(path: str, training: bool):
    global transforms
    if training:
        transforms = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip()
        ])
    else:
        transforms = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor()
        ])

    data = torchvision.datasets.ImageFolder(
        path, transform=transforms)

    data_loader = DataLoader(data, batch_size=32, shuffle=training)

    return data_loader

In [24]:
def train_model(model=Classifier(in_channels=3, num_classes=4)):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    EPOCHS = 30
    train_loader = get_dataloaders(path='Mask_Data/train', training=True)
    model.train()
    for epoch in tqdm(range(EPOCHS)):
        # for batch_idx, (inputs, labels) in enumerate(train_loader):
        for batch in train_loader:
            inputs, labels = batch

            # Forward pass
            y_pred = model.forward(inputs)
           # print('y_pred shape: ', y_pred.shape)

            # Calculating loss
            loss = criterion(input=y_pred, target=labels)

            # BackPropagation
            loss.backward()
            optimizer.step()

            # Zero Gradients
            optimizer.zero_grad()

        print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
    return model


model = train_model()

  3%|▎         | 1/30 [00:06<03:22,  7.00s/it]

Epoch: 0, Loss: 0.3029414117336273


  7%|▋         | 2/30 [00:14<03:29,  7.49s/it]

Epoch: 1, Loss: 0.06849586218595505


 10%|█         | 3/30 [00:23<03:37,  8.06s/it]

...

Epoch: 27, Loss: 0.05811906233429909


 97%|█████████▋| 29/30 [03:33<00:07,  7.01s/it]

Epoch: 28, Loss: 9.530442912364379e-05


100%|██████████| 30/30 [03:40<00:00,  7.36s/it]

Epoch: 29, Loss: 0.0002271579869557172


In [25]:
model

Classifier(
  (layers): Sequential(
    (0): ConvolutionBlock(
      (convs): Sequential(
        (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): ReLU()
      )
    )
    (1): ConvolutionBlock(
      (convs): Sequential(
        (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): ReLU()
      )
    )
    (2): ConvolutionBlock(
      (convs): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      